<a href="https://colab.research.google.com/github/fatima448/Emotion_Recognition_project/blob/main/Emotion_Recognition_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/facial expression recognition.v1i.yolov8.zip"
extract_path = "/content/dataset_yolo"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted to:", extract_path)


Dataset extracted to: /content/dataset_yolo


In [ ]:
import os
import shutil

base = "/content/dataset_yolo"

class_map = {
    "0": "angry",
    "1": "happy",
    "2": "neutral",
    "3": "sad"
}

# New structured dataset for CNN classification
output_dir = "/content/classification_dataset"
os.makedirs(output_dir, exist_ok=True)

for cls in class_map.values():
    os.makedirs(f"{output_dir}/{cls}", exist_ok=True)

print("Created classification folders:", os.listdir(output_dir))


Created classification folders: ['neutral', 'happy', 'angry', 'sad']


In [ ]:
import glob

image_paths = glob.glob(base + "/**/images/*.jpg", recursive=True)
label_paths = glob.glob(base + "/**/labels/*.txt", recursive=True)

label_dict = {}

# Read YOLO label files and extract class IDs
for label_file in label_paths:
    with open(label_file, "r") as f:
        line = f.readline().strip()
    class_id = line.split(" ")[0]  # first value in YOLO label

    # image with same name
    img_file = label_file.replace("labels", "images").replace(".txt", ".jpg")

    # store mapping
    label_dict[img_file] = class_map.get(class_id)

# Copy images to classification folders
count = 0
for img_path, cls in label_dict.items():
    if cls:
        shutil.copy(img_path, f"{output_dir}/{cls}/")
        count += 1

print("Conversion complete! Total images copied:", count)


Conversion complete! Total images copied: 3345


In [ ]:
import cv2
import numpy as np

IMG_SIZE = 48
X = []
y = []
classes = ["angry", "happy", "neutral", "sad"]

for idx, cls in enumerate(classes):
    folder = f"{output_dir}/{cls}"
    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)

        img = cv2.imread(img_path)
        if img is None:
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0

        X.append(img)
        y.append(idx)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

print("Dataset shape:", X.shape)
print("Labels shape:", y.shape)


Dataset shape: (3345, 48, 48, 1)
Labels shape: (3345,)


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# 90% train, 10% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.10, stratify=y, random_state=42
)

# One-hot encoding
y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

print("Train:", X_train.shape, y_train.shape)
print("Test:", X_test.shape, y_test.shape)


Train: (3010, 48, 48, 1) (3010, 4)
Test: (335, 48, 48, 1) (335, 4)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

datagen.fit(X_train)


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0005)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)


In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=64,
    validation_split=0.10
)



Epoch 1/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 12s 228ms/step - accuracy: 0.2668 - loss: 1.3884 - val_accuracy: 0.3920 - val_loss: 1.3489
Epoch 2/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 8s 179ms/step - accuracy: 0.3618 - loss: 1.3379 - val_accuracy: 0.3920 - val_loss: 1.2666
Epoch 3/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 9s 221ms/step - accuracy: 0.3988 - loss: 1.2811 - val_accuracy: 0.4286 - val_loss: 1.2381
Epoch 4/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 9s 206ms/step - accuracy: 0.3994 - loss: 1.2555 - val_accuracy: 0.4286 - val_loss: 1.1760
Epoch 5/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 8s 185ms/step - accuracy: 0.4358 - loss: 1.2179 - val_accuracy: 0.4186 - val_loss: 1.1549
Epoch 6/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 9s 221ms/step - accuracy: 0.4590 - loss: 1.1486 - val_accuracy: 0.4385 - val_loss: 1.2091
Epoch 7/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 8s 178ms/step - accuracy: 0.4739 - loss: 1.1552 - val_accuracy: 0.4684 - val_loss: 1.1216
Epoch 8/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 9s 215ms/step - accuracy: 0.5164 - loss: 1.0992 - val_accuracy: 0

In [ ]:
history_more = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=64,
    validation_split=0.10
)


Epoch 1/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 12s 268ms/step - accuracy: 0.7709 - loss: 0.5328 - val_accuracy: 0.5482 - val_loss: 1.1584
Epoch 2/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 17s 181ms/step - accuracy: 0.8167 - loss: 0.4652 - val_accuracy: 0.5216 - val_loss: 1.1967
Epoch 3/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 9s 212ms/step - accuracy: 0.8313 - loss: 0.4185 - val_accuracy: 0.5382 - val_loss: 1.2565
Epoch 4/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 9s 219ms/step - accuracy: 0.8418 - loss: 0.4044 - val_accuracy: 0.5216 - val_loss: 1.3229
Epoch 5/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 8s 179ms/step - accuracy: 0.8674 - loss: 0.3593 - val_accuracy: 0.5515 - val_loss: 1.3935
Epoch 6/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 9s 220ms/step - accuracy: 0.8920 - loss: 0.3037 - val_accuracy: 0.5382 - val_loss: 1.4110
Epoch 7/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 10s 224ms/step - accuracy: 0.8910 - loss: 0.2856 - val_accuracy: 0.5482 - val_loss: 1.6161
Epoch 8/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 8s 179ms/step - accuracy: 0.9086 - loss: 0.2663 - val_accuracy:

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_acc)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5261 - loss: 1.7905
Test Accuracy: 0.5402985215187073
